In [1]:
import pandas as pd

In [2]:
customer = pd.read_csv("Customer_Master.csv")
transaction = pd.read_csv("transactions.csv")

In [3]:
customer.head(2)

,customer_id,age_group,home_location,credit_score,account_age_years,account_type,avg_monthly_income,mobile_banking_user,primary_device,primary_os,primary_browser,avg_transaction_amount,transaction_frequency,employment_status,preferred_transaction_types,international_activity,risk_score
0,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,Samsung Internet,6498.703229,22,Self-employed,"['Loan Payment', 'Internet Bill Payment', 'Wat...",True,30
1,2,26-35,Kathmandu,607,7,Savings,34640.469961,True,Android Tablet,Android,Chrome,3946.766234,6,Employed,"['Loan Payment', 'Branch Deposit', 'QR Payment...",True,33


In [4]:
transaction.head(2)

,transaction_id,customer_id,transaction_date,transaction_type,amount,location,ip_address,device,os,browser,...,credit_score,account_age_years,account_type,avg_monthly_income,mobile_banking_user,transaction_frequency,employment_status,international_activity,risk_score,is_suspicious
0,TXN20241124104326,727,11/24/2024 15:29,Inward Remittance,13925.72,Nawalparasi,103.92.86.139,NaN,NaN,NaN,...,673,1,Current,65911.89284,True,20,Employed,False,24,False
1,TXN20241204130277,539,12/4/2024 5:26,ATM Withdrawal,25037.35,Kaski,27.201.191.54,NaN,NaN,NaN,...,666,11,Current,75126.72458,True,3,Employed,False,19,False


#### merge these datasets

In [5]:
dataset = customer.merge(transaction, on="customer_id", how="inner")

In [7]:
dataset.head(5)

,customer_id,age_group_x,home_location_x,credit_score_x,account_age_years_x,account_type_x,avg_monthly_income_x,mobile_banking_user_x,primary_device,primary_os,...,credit_score_y,account_age_years_y,account_type_y,avg_monthly_income_y,mobile_banking_user_y,transaction_frequency_y,employment_status_y,international_activity_y,risk_score_y,is_suspicious
0,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,714,13,Savings,55580.82584,True,22,Self-employed,True,30,False
1,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,714,13,Savings,55580.82584,True,22,Self-employed,True,30,False
2,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,714,13,Savings,55580.82584,True,22,Self-employed,True,30,False
3,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,714,13,Savings,55580.82584,True,22,Self-employed,True,30,False
4,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,714,13,Savings,55580.82584,True,22,Self-employed,True,30,False
